In [46]:
%pip install faiss-cpu
%pip install torch transformers sentence-transformers
%pip install pandas numpy beautifulsoup4 scrapy fastapi uvicorn requests


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [48]:
import pandas as pd
import json
import numpy as np
import faiss
import pickle
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer


In [49]:
def load_csv_data(filepath):
    df = pd.read_csv(filepath)
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    df.dropna(inplace=True)
    return df

csv_file_path = "/Users/simarmehta/Downloads/Sample Ingredients File - MenuData Mission.csv"
data = load_csv_data(csv_file_path)
print(data.head())


  restaurant_name menu_category   item_id       menu_item  \
0         20 spot      no proof  24932147  "amaro" spritz   
1         20 spot      no proof  24932146   "gin & tonic"   
2         20 spot      no proof  24932145   amalfi spritz   
3         20 spot      no proof  24932145   amalfi spritz   
5         20 spot      no proof  24932150    blood orange   

          menu_description   ingredient_name  confidence  \
0  pathfinder amaro, tonic  pathfinder amaro        0.95   
1                   lyre's               gin        0.80   
2                   lyre's     amalfi spritz        0.95   
3                   lyre's            lyre's        0.80   
5           san pellegrino      blood orange        0.90   

               categories      address1           city  zip_code country  \
0  New American|Wine Bars  3565 20th St  San Francisco   94110.0      US   
1  New American|Wine Bars  3565 20th St  San Francisco   94110.0      US   
2  New American|Wine Bars  3565 20th St  San

In [50]:
df = pd.read_csv(csv_file_path)
df.columns = [col.strip() for col in df.columns]

def row_to_sentence(row):
    sentence = (
        f"Restaurant: {row['restaurant_name']}. "
        f"Menu Category: {row['menu_category']}. "
        f"Item ID: {row['item_id']}. "
        f"Menu Item: {row['menu_item']}. "
        f"Description: {row['menu_description']}. "
        f"Ingredient: {row['ingredient_name']}. "
        f"Confidence: {row['confidence']}. "
        f"Categories: {row['categories']}. "
        f"Address: {row['address1']}, {row['city']}, {row['state']} {row['zip_code']}, {row['country']}. "
        f"Rating: {row['rating']} based on {row['review_count']} reviews. "
        f"Price: {row['price']}."
    )
    return sentence

df['sentence'] = df.apply(row_to_sentence, axis=1)
print("Example sentence from CSV:")
print(df['sentence'].iloc[0])

output_file_path = 'data_sentences.jsonl'
with open(output_file_path, 'w') as outfile:
    for _, row in df.iterrows():
        record = {
            "item_id": row['item_id'],
            "sentence": row['sentence']
        }
        json.dump(record, outfile)
        outfile.write("\n")
print(f"Sentences have been saved to {output_file_path}.")


Example sentence from CSV:
Restaurant: 20 spot. Menu Category: no proof. Item ID: 24932147. Menu Item: "amaro" spritz. Description: pathfinder amaro, tonic. Ingredient: pathfinder amaro. Confidence: 0.95. Categories: New American|Wine Bars. Address: 3565 20th St, San Francisco, CA 94110.0, US. Rating: 4.3 based on 270.0 reviews. Price: $$.
Sentences have been saved to data_sentences.jsonl.


In [ ]:
import json
import re
from collections import defaultdict

input_file = "data_sentences.jsonl"  

def extract_restaurant_name(sentence):
    """
    Extracts the restaurant name from a sentence, for example:
      'Restaurant: 20 spot. Menu Category: no proof...'
    This function returns '20 spot'.
    """
    match = re.search(r"Restaurant:\s(.*?)\.", sentence)
    if match:
        return match.group(1).strip()
    return "Unknown"

#  map {restaurant_name: [list_of_records]}
grouped_data = defaultdict(list)

with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
        line = line.strip()
        if not line:
            continue 
        
        record = json.loads(line)
        
        sentence = record.get("sentence", "")
        restaurant_name = extract_restaurant_name(sentence)
        
        grouped_data[restaurant_name].append(record)

grouped_data = dict(grouped_data)
output_file_path="output.json"

with open(output_file_path, 'w', encoding='utf-8') as outfile:
    json.dump(grouped_data, outfile, indent=2)

print(f"Grouping complete! Check your output file at: {output_file_path}")


Grouping complete! Check your output file at: output.json


In [54]:
import json
import faiss
import pickle
from sentence_transformers import SentenceTransformer

json_file_path = 'output.json'        
embedding_model_name = 'all-MiniLM-L6-v2'
faiss_index_file = 'faiss_index.bin'


data_records = []
with open(json_file_path, 'r', encoding='utf-8') as infile:
    grouped_data = json.load(infile)  
    
    for restaurant_name, records_list in grouped_data.items():
        for record in records_list:
            data_records.append(record)

sentences = [record['sentence'] for record in data_records]

model = SentenceTransformer(embedding_model_name)
embeddings = model.encode(sentences, convert_to_numpy=True).astype('float32')

embedding_dim = embeddings.shape[1]
# faiss.normalize_L2(embeddings)  # Normalize embedddings for cosine similarity (another vector search mechanism to fetch simialr results)
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)
print(f"FAISS index contains {index.ntotal} vectors.")

faiss.write_index(index, faiss_index_file)
print(f"FAISS index saved to {faiss_index_file}.")

with open('metadata_mapping.pkl', 'wb') as f:
    pickle.dump(data_records, f)
print("Metadata mapping saved to metadata_mapping.pkl.")


FAISS index contains 52696 vectors.
FAISS index saved to faiss_index.bin.
Metadata mapping saved to metadata_mapping.pkl.


In [55]:
def scrape_websites_to_json(urls, output_filename):
    all_data = []
    for url in urls:
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.title.string.strip() if soup.title else "No Title Found"
            paragraphs = [p.get_text(strip=True) for p in soup.find_all('p')]
            page_data = {
                "url": url,
                "title": title,
                "paragraphs": paragraphs
            }
            all_data.append(page_data)
        except requests.exceptions.RequestException as e:
            print(f"Could not scrape {url}: {str(e)}")
            all_data.append({"url": url, "error": str(e)})
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)

urls_to_scrape = [
    "https://www.amenify.com/blog/a-culinary-world-tour-exploring-different-types-of-cuisines",
    "https://en.wikipedia.org/wiki/History_of_pizza",]
output_file = "multiples_scraped_data.json"
scrape_websites_to_json(urls_to_scrape, output_file)


In [56]:
def main():
    json_filename = "multiples_scraped_data.json"
    with open(json_filename, "r", encoding="utf-8") as f:
        data = json.load(f)

    all_paragraphs = []
    metadata_list = []
    for entry in data:
        paragraphs = entry.get("paragraphs", [])
        url = entry.get("url", "")
        title = entry.get("title", "")
        for p in paragraphs:
            all_paragraphs.append(p)
            metadata_list.append((url, title, p))

    if not all_paragraphs:
        print("No paragraphs found in the JSON.")
        return

    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(all_paragraphs, show_progress_bar=True).astype("float32")

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    print(f"FAISS index created with {index.ntotal} vectors.")

    mapping = {}
    for i, (url, title, paragraph) in enumerate(metadata_list):
        mapping[i] = {
            "url": url,
            "title": title,
            "paragraph": paragraph
        }

    faiss_index_filename = "faiss_external.index"
    faiss.write_index(index, faiss_index_filename)
    print(f"FAISS index saved to '{faiss_index_filename}'.")

    mapping_filename = "faiss_external_mapping.json"
    with open(mapping_filename, "w", encoding="utf-8") as f:
        json.dump(mapping, f, ensure_ascii=False, indent=4)
    print(f"Mapping saved to '{mapping_filename}'.")

main()


Batches: 100%|██████████| 5/5 [00:00<00:00, 11.78it/s]


FAISS index created with 130 vectors.
FAISS index saved to 'faiss_external.index'.
Mapping saved to 'faiss_external_mapping.json'.
